## Pair Programming - Regresión logística 6

### Random Forest

---

In [ ]:
#librerías

**Objetivos**:
### 1. Ajustad un modelo de Random Forest a nuestros datos.

### 2. Calculad las métricas a nuestro nuevo modelo.

### 3. Comparad las métricas con los modelos hechos hasta ahora. ¿Cuál es mejor?